# Create dataset for stf

In [ ]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
import pickle
import csv

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    TrainingArguments
)
from accelerate import Accelerator


from trl import SFTConfig, SFTTrainer


#LORA
from peft import LoraConfig, PeftModel

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
def process_pdf(file_path, chunk_size=800, chunk_overlap=50):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-small"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr", 
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases


In [ ]:
base_directory = "./data"
dataset_directory = base_directory + "/train.csv"
df = pd.read_csv(dataset_directory)
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

# PDF 데이터베이스를 pickle 파일로 저장
with open('pdf_databases_cpu.pickle', 'wb') as f:
    pickle.dump(pdf_databases, f)

print("pdf_databases가 pickle 파일로 저장되었습니다.")

In [ ]:
base_directory = "./data"
dataset_directory = base_directory + "/train.csv"
df = pd.read_csv(dataset_directory)
with open('pdf_databases_cpu.pickle', 'rb') as f:
    pdf_databases = pickle.load(f)

In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# 배치 사이즈 설정
batch_size = 1  # 원하는 배치 크기로 설정

# DataFrame의 각 행에 대해 처리
for start in tqdm(range(0, len(df), batch_size), desc="Creating Q&A including RAG info"):
    # 현재 배치 선택
    batch_rows = df.iloc[start:start + batch_size]

    # 배치 내의 각 행 처리
    for _, row in batch_rows.iterrows():
        # 소스 문자열 정규화
        source = normalize_string(row['Source'])
        question = row['Question']
        answer = row['Answer']

        # 정규화된 키로 데이터베이스 검색
        normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
        retriever = normalized_keys[source]['retriever']

        # RAG 체인 구성
        template = """
        다음 정보를 바탕으로 질문에 답하세요:
        {context}

        질문: {question}

        답변:
        """
        prompt = PromptTemplate.from_template(template)
        
        # RAG 체인 정의
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
        )

        # RAG 정보 포함한 prompt 생성
        full_response = rag_chain.invoke(question)

        results.append({
            "Question": full_response.text,
            "Answer": answer
        })


In [ ]:
stf_train_df = pd.DataFrame(results[:int(len(results)*0.8+0.5)])
stf_eval_df = pd.DataFrame(results[int(len(results)*0.2+0.5):])

stf_train_df.to_csv("data/stf_train.csv", index=False, encoding="UTF-8")
stf_eval_df.to_csv("data/stf_eval.csv", index=False, encoding="UTF-8")

In [ ]:
stf_train = pd.read_csv("data/stf_train.csv")
print(stf_train.Question[0])